In [3]:
!pip install transformers

In [8]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)

text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [2]:
import requests


from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import time

model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
#model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:1",
        skip_special_tokens= True, 
        use_triton=use_triton,        
        quantize_config=None)

logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",    
    do_sample=True,
    top_k=10,
    temperature=0.01,
    max_length=1000,
    repetition_penalty=1.1,  
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,)



In [52]:
import json


from nltk.tokenize import sent_tokenize
def tokenize_into_sentences(paragraph):
    sentences = sent_tokenize(paragraph)
    return sentences






    
class LLMStanceDetector:
    def __init__(self, model: str = "mixtral:8x7b",
                 url: str = "http://diceemb.cs.upb.de:8000/api/generate"):
        self.model = model
        self.url = url

    def get_response_from_api_call(self, text: str, claim: str):
        """
        :param text: String representation of an OWL Class Expression
        """
        # prompt=(f"<s> [INST] You are an expert in stance detection. "
        #         f"You have only 3 options (REFUTES, SUPPORTS, and NOT ENOUGH INFO) to detect the stance from the following textual description representing the given claim."
        #         f"[/INST] Model answer</s> [INST] "
        #         f"Given claim: {claim}."
        #         f"Textual description: {text}."
        #         f"Answer only a single option from the three: (1) REFUTES, (2) SUPPORTS, and (3) NOT ENOUGH INFO. "
        #         f"Provide no explanations or write no notes.[/INST]")
        prompt=(f"<s> [INST] You are an expert in stance detection. "
                f"You have only 3 options (REFUTES, SUPPORTS, and NOT ENOUGH INFO) to detect stance from a textual description representing the given claim. "
                f"Please answer only a single option from the three: (1) REFUTES, (2) SUPPORTS, and (3) NOT ENOUGH INFO. "
                f"[/INST] Model answer</s> [INST]\n Given textual description is: {text} \nThe given claim is: {claim}.\n "
                f"You should not output more than 1 word, i.e., (1) REFUTES, (2) SUPPORTS, or (3) NOT ENOUGH INFO. Provide no explanations or write no notes.[/INST]\n")
        # print(prompt)
        response = requests.get(url=self.url,
                                headers={"accept": "application/json", "Content-Type": "application/json"},
                                json={"model": self.model, "prompt": prompt})
        # print(response.json()["response"])
        return response.json()["response"]

    def get_triples_from_transformer(self, text: str, claim: str):
        """
        :param text: String representation of an OWL Class Expression
        """

        prompt=(f"<s> [INST] You are an expert in linguistics and stance detection. "
                    f"Given claim: {claim}."
                    f"You have only 3 options (REFUTES, SUPPORTS, and NOT ENOUGH INFO) to detect the stance from the following textual description representing the given claim."
                    f"Textual description: {text}."
                    f"[/INST] Model answer</s> [INST] "
                    f"Answer only a single option from the three: (1) REFUTES, (2) SUPPORTS, and (3) NOT ENOUGH INFO. "
                    f"Provide no explanations or write no notes.[/INST]")

        # print(prompt)
        print("--------------------------------------------------------------LLM Starts here!!!--------------------------------------------")
    
        response = pipe(prompt)[0]['generated_text']

        response = response.split("[/INST]")[-1]
        
        # response = response.split(" (")[1]
        return response

    
    
verbalizer = LLMStanceDetector()



claim = "Rick Ross had nothing to do with the Maybach Music Group."
text = """ William Leonard Roberts II (born January 28, 1976), known professionally as Rick Ross, is an American rapper and record executive. An influential figure in modern hip hop music, Ri
ck Ross has become known for his "booming" vocal performance, "larger than life" persona, and vivid lyrical imagery. His lyrics form the hardships of street life and black market e
conomic activity into a rags to riches narrative, often describing affluence, wealth, and luxury.Prior to releasing his debut single, "Hustlin'" in March 2006, Ross was the subject
 of a bidding war—receiving offers from Diddy's Bad Boy Entertainment and Irv Gotti's Murder Inc.—ultimately signing a multimillion-dollar deal with Jay-Z on Def Jam Recordings. In
 August of that year, he released his debut studio album Port of Miami, which debuted at the top spot on the Billboard 200 chart, likewise with his second and third studio albums, 
Trilla (2008) and Deeper Than Rap (2009). His fourth album, Teflon Don (2010) spawned the single "Aston Martin Music" (featuring Drake and Chrisette Michele)—which became certified
 triple platinum by the Recording Industry Association of America (RIAA)—while his fifth and sixth albums, God Forgives, I Don't (2012) and Mastermind (2014) peaked the Billboard 2
00 twice more.Outside of his solo career, Ross was a lead member of the hip hop group Triple C's, alongside fellow Florida rappers Gunplay and Torch in 2005. Ross founded the recor
d label Maybach Music Group (MMG) in 2009, and has went on to sign artists including Meek Mill, Wale, French Montana, and Omarion, among others. Maybach Music Group would also ente
r a joint-venture deal with Def Jam and later Epic Records to release his subsequent studio albums: Hood Billionaire (2014), Black Market (2015), Rather You Than Me (2017), Port of
 Miami 2 (2019), and Richer Than I Ever Been (2021). In November 2023, he released a collaborative album with Meek Mill, Too Good to Be True. Ross has been nominated for a total of
 nine Grammy Awards, and has been named "Man of the Year" for two consecutive years by hip hop magazine The Source in 2010 and 2011, as well as "Hottest MC in the Game" by MTV in 2
011. Early life
William Leonard Roberts II was born in Clarksdale, Mississippi, and raised in Carol City, Florida. After graduating from Miami Carol City Senior High School, he attended the histor
ically black college Albany State University on a football scholarship. Roberts worked as a correctional officer for 18 months from December 1995, until his resignation in June 199
7. Career
Career beginnings
In his early years at Suave House Records, Roberts initially made his debut under the pseudonym Teflon Da Don. He made his recording debut on the song "Ain't Shhh to Discuss" on Er
ick Sermon's lone album for DreamWorks, Def Squad Presents Erick Onasis. In the mid-2000s, he changed his name to Rick Ross. He derived his stage name from the former drug kingpin 
"Freeway" Rick Ross, to whom he has no connection.After being signed to Suave House Records, former label for rap duo 8Ball & MJG, he eventually signed a deal with Slip-n-Slide Rec
ords, which has been under the Def Jam label since 2006. While signed to Slip-n-Slide, Roberts toured with fellow rapper Trick Daddy and made guest performances on other Slip-n-Sli
de albums. 2006–2008: Port of Miami and Trilla
His debut album Port of Miami was released in August 2006 and debuted at the top spot on the U.S. Billboard 200 album chart, with sales at 187,000 units during the first week. Chri
stian Hoard of Rolling Stone magazine predicted that it would be "the summer's biggest rap record". The second single was "Push It", which samples "Scarface (Push It to the Limit)"
, the theme song from the gangster film Scarface. The music video for "Push It" was modeled after the film. During that time, Ross made guest performances on two singles from DJ Kh
aled's debut Listennn... the Album: "Born-N-Raised" and "Holla at Me". Port of Miami received Gold certification from the Recording Industry Association of America on November 8, 2
006. In March 2008, his second album Trilla was released and, as its predecessor Port of Miami had, debuted at the top of the Billboard 200. Its lead single "Speedin'" featuring R.
 Kelly peaked at number 21 on the U.S. Billboard Bubbling Under Hot 100 Singles; the next one, "The Boss" featuring T-Pain peaked at number 17 on the Hot 100. The third single "Her
e I Am" featured Nelly and Avery Storm. MTV News ranked Ross on the fourth spot in its 2008 "Hottest MCs in the Game" list among ten rappers. The fourth single "This Is The Life" f
eatured Trey Songz and was released in July. 2009: Deeper Than Rap
I look at the game and the business and all different aspects, it's a lot of great lyricists on the corner that will never properly understand the business and know how to market t
hemselves and get in a position where they can gain capital. I look at all the strategies people use and what made them successful. What made Birdman just as relevant today after s
elling 50 million records? That intrigues me. To see the class of Jay-Z, his accomplishments and see how he sits backs and accurately makes his moves. A track from the album called
 "Valley of Death" was what stood out to MTV reporters. In the song, Ross speaks briefly on his controversial stint as a prison guard. "Keep it trilla, nigga, never had a gun and b
adge," which he stresses, leaving the word that he was indeed an officer of the law. "Kept a nice watch, smoking on a hundred sack/ Back in the day I sold crack for some nice kicks
/ Skippin' school, I saw my friend stabbed with an ice pick/ Can't criticize niggas trying to get jobs/ Better get smart, young brother, live yours." Later, he implies that while h
e was working as a corrections officer, he was on the streets. "Only lived once and I got two kids/ And for me to feed them, I'll get two gigs," he raps. "I'll shovel shit, I'll C.
O./ So we can bow our heads and pray over the meatloaf." He still provided no explanation for lying about being a C.O. to begin with, nor did he explain why he failed to pay child 
support for his children.The cover of the May 2009 issue of XXL magazine, titled "Rick Ross Up in Smoke", featured Ross wearing a pair of Louis Vuitton sunglasses. After publicatio
n, a spokesperson for the luxury-goods maker contacted XXL to inform them that Ross was sporting fake sunglasses in the cover image. It was later revealed that the rapper was weari
ng authentic Louis Vuitton sunglasses which were altered by Jacob Bernstein, known as "The Sunglass Pimp". Bernstein defended his customizations despite Louis Vuitton's insistence 
that such modification and resale of trademarked property is not legal. 2010–2013: Prime Rick Ross
Ross had a number of accomplishments in 2010. In May, he released the critically acclaimed Albert Anastasia EP. In addition to that, he released his fourth studio album entitled Te
flon Don, on July 20, 2010. The lead single was "Super High" featuring Ne-Yo. Actress Stacey Dash made an appearance in the music video. A second official single was released title
d "Live Fast, Die Young" which features and is produced by Kanye West. The album debuted at number two on the Billboard 200 selling 176,000 copies in its first week. In November 20
10, rapper and frequent collaborator, Diddy, who in 2009 signed Ross to a management deal with his Ciroc Entertainment, announced from his YouTube account, that he and Ross are wor
king on an EP together. A single was released for the project, called "Another One". Ross ended 2010 by releasing another mixtape, Ashes To Ashes on Christmas Eve 2010. In early 20
11, Rick Ross signed Wale and Meek Mill to his Maybach Music Group imprint. The group released their debut compilation album, Self Made Vol. 1 in May. Rick Ross performed at BET Aw
ards 2011, which aired on June 26, 2011. He also received a nomination for Best Male Hip Hop Artist. In early 2012, Ross was named the Hottest MC in the Game.Big things were happen
ing for Ross's successful career in 2012 as well. He started the year by premiering his Rich Forever mixtape to universal acclaim. Ross's MMG camp released their second compilation
, Self Made Vol. 2 in June 2012. Less than a month later, Ross's fifth studio album, God Forgives, I Don't, was released on July 31, 2012, after a few delays. Upon its release, the
 album charted at number eight on the UK Albums Chart and number two on the UK R&B Albums Chart, making it Ross's highest-charting album and first top 10 album in the United Kingdo
m. The album debuted at number one on the Billboard 200, with first-week sales of 218,000 copies.Ross premiered four songs: "So Sophisticated" featuring Meek Mill, "Touch'N You" fe
aturing Usher, "Hold Me Back", and "3 Kings" featuring Dr. Dre and Jay-Z. The album was certified gold in less than two months. To celebrate his accomplishment and promote his upco
ming MMG Tour, Ross released a mixtape, The Black Bar Mitzvah in October. Rick Ross was nominated by The Source as its "Man of the Year".On January 7, 2013, Ross revealed the title
 of his sixth album to be Mastermind. The album was set to be released in 2013. The first promotional single from the album would come in the form of "Box Chevy" which was released
 to iTunes on February 15, 2013. The music video was filmed on April 1, 2013, and features cameos from MMG members Gunplay, Stalley and Rockie Fresh.During 2008, it was announced t
hat Ross was set to release a collaboration album with his longtime friend Birdman titled The H, but after that, no information on the album was released. On May 16, 2013, Birdman 
and Rick Ross announced that The H would be released as a mixtape hosted by DJ Khaled on May 23, 2013. The project was recorded during a couple of days in 2008.On September 5, 2013
, Ross premiered the first official single from Mastermind, "No Games" featuring Future produced by J.U.S.T.I.C.E. League. The following day, it was released to Urban contemporary radio. It was followed by the singles "The Devil Is A Lie" with Jay-Z and "War Ready" with Jeezy. 2014: Mastermind and Hood Billionaire
Rick Ross released his sixth studio album, Mastermind, in March 2014, the album received generally positive reviews from critics and debuted at number one on the Billboard 200 chart with first week sales of 179,000 copies. As of April 8, 2014, the album has sold over 290,000 copies in the United States. In June 2014, it was announced that Ross would make a guest appearance on the docu-series Sisterhood of Hip Hop.In October 2014, Ross announced he would be releasing another album, Hood Billionaire, in order to make up for Meek Mill's Dreams Worth More Than Money album being shelved due to his incarceration. Rick Ross revealed cover art for Hood Billionaire in October 2014 and announced the release date to be set for November 24, 2014. The album received mixed reviews, from fans and critics alike. 2015–present: Renzel and Epic Records Ross had a mix of both adversity and accomplishments in 2015. Ross was arrested for kidnapping and assault charges and was sentenced to house arrest in his Georgia home. Ross joined the Snapchat platform and began referring to himself as "Renzel", a play on the name of famous actor Denzel Washington. In September, Ross released a mixtape, Black Dollar, his third full-length release since March 2014. He soon announced his eighth studio album, Black Market, for a December 2015 release. To promote his upcoming release, Ross released a remix tape, Renzel Remixes on Thanksgiving 2015. Black Market was released on December 4, 2015, and it served as Ross's final album for his original contract with Def Jam. The album's song "Free Enterprise" stirred up controversy because the lyrics included the line "Assassinate Trump like I'm Zimmerman" referring to Donald Trump.In January 2016 after much speculation, Rick Ross signed a solo deal with Epic Records, reuniting him with L.A. Reid, who originally played a major role in signing Ross to Def Jam.On February 21, 2016, Rick Ross remixed Kanye West's "Famous". Ross was active in 2016, releasing many songs including "Make It Work" off of Self Made Vol. 4, "Purple Lamborghini" of Suicide Squad: The Album, "Supa Cindy", and "Freaky Hoe". He announced that his ninth studio album, Rather You Than Me, was slated for a March 2017 release. The album debuted and peaked number three on the  Billboard 200. On February 2, 2018, Rick Ross released "Florida Boy" with T-Pain and Kodak Black as the first single from the album, which was slated for a 2018 release. On June 22, 2018, Ross released "Green Gucci Suit" with Future as the second single from his tenth studio album Port of Miami 2, released on August 9, 2019. The album debuted at number two on the Billboard 200.The album included the single "Gold Roses" with Drake, which was nominated for a Grammy Award for Best Rap Song at the 62nd Annual Grammy Awards. In July 2020, Ross teamed up with singer August Alsina for a track titled "Entanglements", a diss track towards Alsina's ex-lover Jada Pinkett Smith.In 2017, Ross was criticized for stating in an interview that he never signed a female rapper because "I would end up fucking [her] and fucking the business up." Ross quickly apologized through a Facebook post where he called his comments "insensitive" and a "mistake [he regrets]. "In August 2020, Ross released the single "Pinned to the Cross", featuring Finn Matthews, following his Verzuz battle with 2 Chainz. 2021: Richer Than I Ever Been
On August 26, 2020, during an interview with Hot 97, Rick Ross announced his eleventh album, Richer Than I Ever Been, which was released on December 10, 2021.On September 7, 2021, Ross' book The Perfect Day to Boss Up debuted at number 5 on The New York Times' weekly bestsellers list. Personal life
Family
Ross has four children: Toie Roberts, born March 2002. William Roberts III, born September 2005. Berkeley Hermès Roberts, born September 2017, Billion Leonard Roberts, born November 2018 and Bliss Roberts, born August 2020.Ross was briefly engaged to Lira "Galore" Mercer in 2015. Health
On October 14, 2011, Ross suffered two seizures in the same day, once in the morning and again in the early evening. Following the morning seizure, he was unconscious and CPR was performed. After suffering the second seizure, Ross was admitted to a hospital in Birmingham, Alabama, where doctors ran a battery of tests. They declared him healthy. Ross attributes the episodes to a lack of sleep in the previous week. Ross later noted the incident in the remix of Meek Mill's song "Ima Boss", with the line "Had a couple seizures, call it minor setbacks / Everybody praying for me, I respect that / Woke up in the hospital, where my checks at? / Then I put 8 chains where my neck's at".On March 2, 2018, Ross was hospitalized after being found unresponsive in his Miami, Florida, home from a possible heart attack. Ross was on life support. He was released from the hospital a few days later. Religion
Rick Ross is a Christian, and in an interview with Corey "Coco Brother" Condrey on Lift Every Voice, he indicated that "I had told myself at one point no matter what I go through, I never question God." In addition, Ross indicated that he prays every time he goes on stage to perform. Correctional officer photos
In July 2008, The Smoking Gun produced details linking Ross's social security number to an 18-month stint as a correctional officer at the South Florida Reception Center, along with a photograph first publicized by Media Take Out purporting to be Ross in his correctional officer uniform. Ross initially denied that the photograph was of him, but after overwhelming evidence of his past came into the public's eye, Ross later admitted that he did work as a correctional officer in Florida for 18 months between 1995 and 1997. In a 2008 interview with AllHipHop.com, "Freeway" Ricky Ross claimed offense that Roberts used his name and identity when he learned that The Smoking Gun posted documents revealing Roberts' previous employment as a Florida correctional officer. Drive-by shooting
On January 27, 2013, while celebrating his 37th birthday, Ross and his girlfriend were the targets of a drive-by shooting in Fort Lauderdale, Florida. Neither Ross nor his girlfriend were hurt, but they ended up crashing their car into an apartment in an attempt to escape the shooting. Business ventures
As of 2017, Ross owned several different Wingstop restaurant locations. Wingstop is a fast-food chain specializing in chicken wings. When Don Lemon was fired from CNN in April 2023, Ross offered him a job, saying, "For some reason, I believe you know what you doing with the lemon pepper wings. So man, just send over your résumé." Legal issues
In January 2008, Ross was arrested on gun and marijuana charges. During a deposition of Officer Rey Hernandez, a Miami Beach police officer who arrested Ross, Ross's lawyer, Allan Zamren, asked Hernandez why Ross's case was assigned to the gang task force. The officer stated that it was because Ross claimed affiliation with Triple C's (aka Carol City Cartel), and other known gang members. Zamren pressed Hernandez for a personal link between Ross and the gang members, but the officer did not provide proof.In August 2008, YouTube entertainer DJ Vlad filed a lawsuit against Ross for assault and battery. Vlad claimed Ross organized an ambush on him at the 2008 Ozone Awards in Houston, Texas for asking questions about his past as a correctional officer.In June 2010 "Freeway" Ricky Ross sued Ross for using his name, filing a copyright infringement lawsuit in a California federal court. Jay-Z had been called to testify in the lawsuit, as he was CEO of Def Jam when Ross/Roberts was signed. "Freeway" Ricky Ross, a reformed drug kingpin, sought $10 million in damages in his lawsuit. Also, Freeway Ricky Ross threatened to block the release of Ross's album, Teflon Don. A week after the lawsuit was filed, Rick Ross responded to the charges: "It's like owning a restaurant, you're gonna have a few slip and falls. You get lawsuits, you deal with them, and get them out your way ... sometimes you lose." He denied rumors that he would change his name to "Ricky Rozay" as a consequence of the lawsuit. The lawsuit was thrown out of court on July 3, 2010. The album Teflon Don was released as scheduled on July 20, 2010. A federal judge ruled that the case should be refiled in California state court because it fell under California state law. "Freeway" Ricky Ross refiled the case with the State of California and the federal case is on appeal in the United States Court of Appeals for the Ninth Circuit. The state case was filed in 2011 in California. "Freeway" Ricky Ross refiled in Los Angeles Superior Court with publicity rights claims. Trial was set for early May 2012. The case was dismissed by a judge in the Los Angeles Superior Court.The California State case was updated with a motion in Freeway Rick Ross's favor as to Warner Bros. Records and their use of the name and image Rick Ross in July 2012.On December 30, 2013, the court ruled in favor of the rapper Rick Ross, allowing him to keep the name based on a First Amendment ruling.In 2011, the rapper Teflon Don filed a trademark infringement lawsuit against Ross for the use of the name as an album title.On March 26, 2011, Ross was arrested in Shreveport for possession of marijuana. According to the police records, a strong odor of marijuana was detected from his room at the Hilton in Downtown Shreveport. Ross was arrested on June 27, 2014, in Greensboro, N.C., following his performance at the 2014 SuperJam at the Greensboro Coliseum. The arrest stemmed from a year-old warrant for another North Carolina arrest related to marijuana possession; Ross reportedly failed to appear in court for that violation in 2013.On June 24, 2015, Rick Ross and his bodyguard were arrested in Fayette County, Georgia on kidnapping and assault charges. Both were accused for forcing a landskeeper into a guesthouse and beating him in the head with a handgun, allegedly over money he owed Ross. On April 4, 2017, Ross pled no contest to a felony count of aggravated assault. The other felony charges were dropped. Ross was sentenced to time served in jail, having already been incarcerated for a week and was also ordered to pay a $10,000 fine. Ross was sentenced as a first-time felony offender; and, upon payment of the fine, the conviction was erased from his record. Ross pled no contest to 5 misdemeanours: assault, battery, two counts of pointing a pistol at someone, and possession of less than 1 oz of cannabis. For the five misdemeanours, he was given a total of 60 months probation. Controversies
Feud with 50 Cent
In January 2009, Ross started a feud with rapper 50 Cent because he supposedly looked at him the wrong way at the BET Hip Hop Awards in 2008. 50 Cent told news sources that he did not remember seeing Ross there. In late January, "Mafia Music", by Ross, leaked onto the Internet. There were several lines that seemed to diss 50 Cent. Days later, 50 Cent released "Officer Ricky (Go Head, Try Me)" in response to Ross's disparaging remarks on his "Mafia Music" song. Before going to Venezuela, 50 Cent uploaded a video entitled "Warning Shot", wherein he stated: "Rick Ross, I'mma fuck your life up for fun." In addition, 50 Cent released the first of a series of "Officer Ricky" cartoons. Early in February, 50 Cent once again made a video which he uploaded to YouTube in which he interviewed "Tia", the mother of one of Ross's children. She verifies that he was a correctional officer and claims his whole persona is fake and fraudulent. On February 5, 2009, The Game, who 50 Cent has a long-standing "beef" with, called up Seattle R&B station KUBE. When asked about the beef between 50 Cent and Ross, Game sided with 50 Cent and said that things are not looking good for Ross. He offered to help, stating, "Rick Ross, holla at your boy, man," and, "50 eating you, boy. "On his album Deeper Than Rap, Ross references 50 Cent in the song "In Cold Blood". A video for the song was released that portrayed 50 Cent's mock funeral. Upon release, Ross stated that he has ended 50 Cent's career. That same day Ross released a new diss track called "Push 'Em Over The Edge". The next day 50 Cent released the controversial video "A Psychic Told Me", dissing DJ Khaled. On February 12, 50 Cent responded with "Tia Told Me", along with Lloyd Banks' response "Officer Down" and Tony Yayo's "Somebody Snitched". In an interview, 50 Cent said Ross is "Albert From CB4. You ever seen the movie? He's Albert," he added. "It never gets worse than this. You get a guy that was a correctional officer come out and base his entire career on writing material from a drug dealer's perspective. "On January 30, 2013, 50 Cent claimed via Twitter that the drive-by shooting that Ross experienced on his birthday two days ago was "staged", reigniting the feud.A year later, on March 28, 2014, Rick Ross was asked about his feud with 50 Cent. He stated that he does not wish to speak on him anymore, and that he is done with the feud.His feud with 50 Cent has been resurfacing. The reason of the second reignition would be because of Ross's MMG record-label artist Meek Mill, who defended his mentor and criticised 50 on his 4/4 EP. Lyric controversy
In a line on rapper Rocko's 2013 song "U.O.E.N.O. ", Rick Ross raps the line, "Put molly all in her champagne / She ain't even know it / I took her home and I enjoyed that / She ain't even know it." A petition containing 72,000 signatures was presented to Reebok, demanding they drop Ross as a spokesman for the lyrics which appeared to condone date rape. Ross has apologized for the lyrics, claiming they weren't about rape.On April 11, 2013, one month after the song's release, Ross was dropped by Reebok. Reebok stated, "while we do not believe that Rick Ross condones sexual assault, we are very disappointed he has yet to display an understanding of the seriousness of this issue or an appropriate level of remorse." They continued on to say, "at this time, it is in everyone's best interest for Reebok to end its partnership with Mr. Ross. "Though Ross apologized for the lyrics, he also argued that his lyrics were misinterpreted because he did not directly use the word "rape." He says, "You know I would never use the term rape in my records ... in reality some people do these things, and shouldn't it be brought to light so young women can protect themselves?!" He continued on Twitter, "I would never use the term 'rape' in my records and as far as my camp. Hip-Hop don't condone that, the streets don't condone that, nobody condones that. "A 2013 Ross concert organized by the student association of Carleton University was cancelled after protests that his lyrics promote rape culture. Rocko later dropped the Rick Ross verse in order to get radio play. Discography
Studio albums

Port of Miami (2006)
Trilla (2008)
Deeper Than Rap (2009)
Teflon Don (2010)
God Forgives, I Don't (2012)
Mastermind (2014)
Hood Billionaire (2014)
Black Market (2015)
Rather You Than Me (2017)
Port of Miami 2 (2019)
Richer Than I Ever Been (2021)Collaborative albums

Custom Cars & Cycles (with Triple C's) (2009)
Too Good to Be True (with Meek Mill) (2023)

Filmography
Awards and nominations
Bet Awards
Grammy Awards
The Grammy Awards are awarded annually by the National Academy of Recording Arts and Sciences of the United States. Rick Ross has nine nominations

The Source
References
External links

Official website
Rick Ross at AllMusic
Rick Ross at IMDb
1
"""
sentences = tokenize_into_sentences(text.replace("\n"," "))
# print(" ".join(sentences[:5]))
# exit(1)
text = " ".join(sentences[:500])
answer = verbalizer.get_response_from_api_call(text=text, claim= claim)
print("answer is:"+ answer)
final_verdict = "NOT ENOUGH INFO"
if "REFUTES".lower() in answer.lower() and "SUPPORTS".lower() not in answer.lower():
    final_verdict = "REFUTES"
    print("ANSWER->REFUTES")
elif "SUPPORTS".lower() in answer.lower() and "REFUTES".lower() not in answer.lower():
    final_verdict = "SUPPORTS"
    print("ANSWER->SUPPORTS")
elif "NOT ENOUGH INFO".lower() in answer.lower():
    print("ANSWER->NOT ENOUGH INFO")

In [47]:

claim_lines = []
# Read data from a file (assuming it's in JSON format)
count =0

with open('output.jsonl', 'r') as output_file:
    for line in output_file:
        cdata = json.loads(line)
        cc = cdata['claim']
        claim_lines.append(cc)
        
with open('output_fever_test.jsonl', 'r') as f,  open('output.jsonl', 'a') as output_file:
    for line in f:
        data = json.loads(line)
        claim = data['claim']
        if claim not in claim_lines:
            claim_lines.append(claim)
            text = data['text']
            print("claim:"+ claim)
            print("ground truth label:"+ data['label'])
            sentences = tokenize_into_sentences(text.replace("\n"," "))
            # print(" ".join(sentences[:5]))
            # exit(1)
            print(len(sentences))
            text = " ".join(sentences[:50])
            answer = verbalizer.get_response_from_api_call(text=text, claim= claim)
            # print("answer is:"+ answer)
            final_verdict = "NOT ENOUGH INFO"
            if "REFUTES".lower() in answer.lower() and "SUPPORTS".lower() not in answer.lower():
                final_verdict = "REFUTES"
                print("ANSWER->REFUTES")
            elif "SUPPORTS".lower() in answer.lower() and "REFUTES".lower() not in answer.lower():
                final_verdict = "SUPPORTS"
                print("ANSWER->SUPPORTS")
            elif "NOT ENOUGH INFO".lower() in answer.lower():
                print("ANSWER->NOT ENOUGH INFO")
                final_verdict = "NOT ENOUGH INFO"
            else:
                print(text)
                print(answer)
                break

            data['prediction'] = final_verdict

            # Write the updated JSON object to the output file
            json.dump(data, output_file)
            output_file.write('\n')

            # print(data)
            print("--------------------------------claim ENDS---------------------------")
            count = count +1
            if count >1000:
                break

# print(lines)
        # exit(1)
    
